In [ ]:
# But first, let's install the dependecies.
!pip install -r requirements.txt

In [1]:
import os
from dotenv import load_dotenv
from video_prompter import get_connection

# TODO: setup .env file
load_dotenv()

# connect to VideoDB
conn = get_connection()

### ----- Upload a fresh video --------- #####
def fresh_video(url):
    video = conn.upload(url=url)
    #index spoken content in the video
    video.index_spoken_words()
    return video

#### ------ run the prompt on video --------####
def videodb_prompter(video_id, prompt):
    video = get_video(video_id)
    #get all the segment of videos that are
    return video_prompter(video, prompt)

In [2]:
from videodb import play_stream

# ----Existing video case ----

# TODO: replace with your video id
# video_id = "null"


# ---- Fresh video case ----
url = "https://youtu.be/pAMy7IhOVQE?si=iNvbJE02Pf8XXGd8"
video = fresh_video(url)
video_id = video.id


#watch the original video
video = conn.get_collection().get_video(video_id)
video.player_url

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/bd38e2b1-fd23-4629-935e-5d38a80738fe.m3u8'

In [ ]:
# Create new scene index
from videodb import IndexType
from videodb import SceneExtractionType
 
index_id = video.index_scenes(
    extraction_type=SceneExtractionType.time_based,
    extraction_config={"time":10, "select_frames": ['first']},
    prompt="Describe in detail the actions, gestures, and movements of each character visible in the frame. Include their position, facial expressions, and any interactions with objects or other characters.",
    # callback_url=callback_url,
)

In [ ]:
# Wait to Indexing to finish
scene_index = video.get_scene_index(index_id)

In [5]:
transcript = video.get_transcript()

In [ ]:
from video_prompter import multimodal_prompter


prompt = "a conversastion on religion"
matches = multimodal_prompter(transcript,scene_index, prompt)


In [9]:
# search matching video segments and watch the stream
from videodb import SearchType
from videodb.timeline import Timeline, VideoAsset, AudioAsset
from video_prompter import extract_clip_sentences


timeline = Timeline(conn)
for clip_sentences in extract_clip_sentences(matches):
    search_res = video.search(clip_sentences, search_type=SearchType.keyword)
    matched_segments = search_res.get_shots()
    
    #no exact match found.
    if len(matched_segments) == 0:
        continue

    #video segment
    video_shot = matched_segments[0]

    #Create a new Video Asset and add it to a timeline.
    timeline.add_inline(VideoAsset(asset_id=video.id, start=video_shot.start, end=video_shot.end))

Processed Item 0


In [10]:
stream = timeline.generate_stream()
print(stream)
play_stream(stream)

https://stream.videodb.io/v3/published/manifests/17712372-e786-4c87-8b2a-c3b4ad9d553e.m3u8


'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/17712372-e786-4c87-8b2a-c3b4ad9d553e.m3u8'